In [1]:
# os and sys
import os
import datetime as dt

# data
import pandas as pd
import numpy as np

In [2]:
# genus species
rand_letters = ["a", "b", "c", "d", "e", "f", "g", "h","i"]
plant = "plant"
number = [x+1 for x in np.arange(len(rand_letters))]

names = [f"{plant}-{x}" for x in number]

In [3]:
# growth times in days

# hibernation
htime = [21, 38, 52, 14, 0]
# germinate
gtime = [10, 15, 21, 30, 18, 28]
# to bloom
btime = [60, 90, 75, 120, 150, 45, 30]

# assign the values randomly based on a probability dist
# less bloom time higher probability
growth_times={}
for plant in names:
    h_t = np.random.choice(htime, size=1, p=[.06, .05,.01,.24,.64])
    g_t = np.random.choice(gtime,size=1, p=[.01, .12, .27, .01, .34, .25])
    b_t = np.random.choice(btime, size=1, p=[.4, .1, .1, .05, .05, .15, .15])    
    g = {plant:{"hibernate":h_t[0], "germinate":g_t[0], "bloomtime":b_t[0]}}
    growth_times.update(g)
    
g_times = pd.DataFrame.from_dict(growth_times, orient='index')

In [4]:
# assign a required space for each plant to reach maturity
b_times = g_times.bloomtime.unique()
b_times = sorted(b_times)

# required space for plant to bloom in:
# ground cover, quick germinate and bloom, supports feature and support plants
g_times['space'] = 1

# anchor plant, the plant that supports multiple types of whatever
g_times.loc[g_times.bloomtime == b_times[-1], 'space'] = int(((50/2)**2)*(3.14))

# feature plants, support the cover and the anchor
g_times.loc[g_times.bloomtime == b_times[-2], 'space'] = int(((20/2)**2)*(3.14))

# support plants, support the feature and the anchor
g_times.loc[g_times.bloomtime == b_times[-3], 'space'] = int(((15/2)**2)*(3.14))

In [5]:
# assign the space to required to bring germinate one plant:

# number of seeds for one plant:
seeds = 1

# space for one seedling prior to transplant cm²

# assign a fail rate
fail_rate=.2

# assign a target value in sq meters
deliver=100

# assign a delivery date
d_date = "2022-03-01"

# assign a through date, you must maintain the installation for that amount of time
thru_date = "2022-09-30"

In [6]:
# build an order based on the bloom time and growth times
# to do that we will base the values of the plant on its frequency or abundance
# the more abundant plants will take up the least space and have the shortest germination times

# assign a category to each plant based on bloom time
g_times['group'] = 'cover'
g_times.loc[g_times.bloomtime == b_times[-1], 'group'] = "anchor"
g_times.loc[g_times.bloomtime == b_times[-2], 'group'] = "feature"
g_times.loc[g_times.bloomtime == b_times[-3], 'group'] = "support"

In [7]:
# assigin a percentage of space for each group or category in the final instalation:
support = .1
anchor=.1
feature=.3
cover=.5

In [8]:
# randomly select the plants for the instalation
s_choices = g_times[g_times.group == 'support'].index.unique()
c_choices = g_times[g_times.group == 'cover'].index.unique()
f_choices = g_times[g_times.group == 'feature'].index.unique()
a_choices = g_times[g_times.group == 'anchor'].index.unique()

In [9]:
# 10,000 cm² = 1m²

# the number of cover plants
n_cover = int(((50*10000)/1))

# the number of support plants
n_supp = int((10*10000)/176)

# the number of feature plants
n_feat = int((30*10000/314))

# the number of anchors
n_anchor= int((10*10000/1962))

In [10]:
# the number of plants to be germinated
# assume the anchors is a perenial
# assume that the support plants are perenial or whatever you don't need to germinate

c_germ = n_cover/(1-fail_rate)
f_germ = n_feat/(1-fail_rate)

# the number of plants you will have to replace for the specified period of time
# count that every plant will have to be replaced once
t_germ = (c_germ + f_germ)*2

# the space required to germinate
# cover plants are like sod or grass you can get many in few centimeters squared:
space_cover = int((c_germ)/10000)

# count 2cm² to germinate each feature plant
space_feature =int((f_germ*2)/10000)

# the space for perenials
space_support = int((n_supp*176)/10000)

# space for anchor
anchor_space = int((n_anchor*1962)/10000)

# space to keep inventory
invspace = space_cover + space_feature+ space_support+anchor_space
total_space = invspace*2

In [11]:
# minimum space required
# plus place to put tools and assume some vertical arrangement

total_space+(total_space*.3)

208.0

In [12]:
# plants to germinate
int(t_germ)

1252387

In [13]:
# seeds by type
print(F"cover: {c_germ}, feature:{f_germ}")

cover: 625000.0, feature:1193.75


In [14]:
# the number of each type of plant needed for inventory and delivery
x = pd.DataFrame.from_dict({'cover':n_cover, 'support':n_supp, 'feature':n_feat, 'anchor':n_anchor},orient='index')
x.rename(columns={0:'n plants'})

,n plants
cover,500000
support,568
feature,955
anchor,50


In [15]:
# your plants:
g_times

,hibernate,germinate,bloomtime,space,group
plant-1,14,15,150,1962,anchor
plant-2,38,15,60,176,support
plant-3,0,18,60,176,support
plant-4,0,18,45,1,cover
plant-5,14,15,30,1,cover
plant-6,0,10,60,176,support
plant-7,0,15,75,314,feature
plant-8,0,21,60,176,support
plant-9,14,28,60,176,support
